# Tree Methods Documentation Examples

### 1. Import koniecznych pakietów oraz wstępne przygotowanie danych.

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mytree').getOrCreate()

In [5]:
from pyspark.ml import Pipeline

Importujemy klasy związane z drzewami decyzyjnymi.

In [6]:
from pyspark.ml.classification import (RandomForestClassifier,
                                      GBTClassifier,
                                      DecisionTreeClassifier)

Import danych:

In [7]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

Zbiór danych został zaczerpnięty z dokumentacji Sparka. Został już podzielony na kolumny 'label' oraz 'features'. Z tego wzgledu nie wymaga już dalszych przygotowań.

In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



Dokonujemy losowego podziału danych na część treningową oraz na część testową.

In [10]:
train_data, test_data = data.randomSplit([0.7, 0.3])

Treningowy zbiór danych:

In [14]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                 75|
|   mean|               0.56|
| stddev|0.49972965664419966|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



Testowy zbiór danych:

In [15]:
test_data.describe().show()

+-------+-----+
|summary|label|
+-------+-----+
|  count|   25|
|   mean|  0.6|
| stddev|  0.5|
|    min|  0.0|
|    max|  1.0|
+-------+-----+



### 2. Tworzenie i trening klasyfikatorów

W pierwszej kolejności stworymy Decistion Tree Classifier, Random Forest Classifier oraz Gradient Boosted Classifier. Nie ma konieczności definiowania parametrów featuresCol oraz labelCol ponieważ ich nazwy są zgodne z wartościami domyślnymi. Jedynym parametrem, który zostanie przez nas zmieniony jest ilość drzew w RFC na równą 100. Ilość użytych drzew poprawia wyniki obiczeń, jednak po przekroczeniu pewnej ilości drzew dodawanie kolejnych nie poprawia już dokładności, za to spowalnia obliczenia.

In [13]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

Po utworzeniu modeli możemy rozpocząć ich trening w oparciu o testowy zbiór danych.

In [16]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

Przewidywanie wartości 'label' przy użyciu wytrenowanych modeli.

In [18]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

Zarówno dtc jak i rfc posiadają kolumnę rawPrediction, w odróżnieniu do gbt, ktre tej kolumny nie posiada. Multiclass Classification Evaluator lub Binary Class Classificator bardzo często mają ustawione jako argument domyślny rawPrediction. Wtedy dla gbt należy to zmienić. 

In [19]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[121,122,123...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,148...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[151,152,153...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[234,235,237...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  1.0|(692,[99,100,101,...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  1.0|(692,[100,101,102...|   [0.0,42.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

In [20]:
gbt_preds.show()

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|(692,[121,122,123...|       0.0|
|  0.0|(692,[122,123,148...|       0.0|
|  0.0|(692,[123,124,125...|       0.0|
|  0.0|(692,[124,125,126...|       0.0|
|  0.0|(692,[124,125,126...|       0.0|
|  0.0|(692,[126,127,128...|       0.0|
|  0.0|(692,[126,127,128...|       0.0|
|  0.0|(692,[151,152,153...|       0.0|
|  0.0|(692,[152,153,154...|       0.0|
|  0.0|(692,[234,235,237...|       0.0|
|  1.0|(692,[99,100,101,...|       0.0|
|  1.0|(692,[100,101,102...|       1.0|
|  1.0|(692,[124,125,126...|       1.0|
|  1.0|(692,[125,126,127...|       1.0|
|  1.0|(692,[126,127,128...|       1.0|
|  1.0|(692,[126,127,128...|       1.0|
|  1.0|(692,[126,127,128...|       1.0|
|  1.0|(692,[127,128,129...|       1.0|
|  1.0|(692,[127,128,155...|       1.0|
|  1.0|(692,[129,130,131...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



### 3. Ewaluacja

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Oszacowanie dokładności za pomocą MulticlassClassificationEvaluator.

In [22]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [27]:
print('DTC ACCURACY:')
acc_eval.evaluate(dtc_preds)

DTC ACCURACY:


0.96

In [30]:
print('RFC ACCURACY:')
acc_eval.evaluate(rfc_preds)

RFC ACCURACY:


0.96

In [32]:
print('GBT ACCURACY:')
acc_eval.evaluate(gbt_preds)

GBT ACCURACY:


0.96

### 4. Feature importance

In [34]:
#rfc_model.featureImportances